In [1]:
import pandas as pd
from h3 import h3
import folium

In [48]:
#функция создания словаря гексов для отображения на карте из xlsx
def xlsx_to_dict(file):
    df = pd.read_excel(file)
    header = list(df)
    dict_list = [] #список словарей
    for i in range (df[header[1]].count()):
        dict_list.append(dict(zip(header, df.iloc[i])))
    
    return dict(zip(df[header[0]], dict_list)) 

In [60]:
def visualize_hexagons(hexagons_dict, color='#bb111166', folium_map=None):
    hexagons = list(hexagons_dict.keys())
    polylines = {}
    lat = []
    lng= []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0] #список линий, которые формируют гекс
        lat.extend(map(lambda v:v[0], polyline)) #все широты в один список, для нахождения средней точки
        lng.extend(map(lambda v:v[1], polyline)) 
        polylines[hex] = polyline

    if folium_map is None:
            m = folium.Map(locations=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=8, tiles='cartodbpositron')
    else:    
            m = folium_map 
    for polyline in polylines.items():
            my_Polyline = folium.PolyLine(locations = polyline[1],
                                          weight = 3,
                                          color = color,
                                          fill = True,
                                          fill_color = "red",                                          
                                          popup = 'id гекса: ' + polyline[0] + '\n\nКол-во жителей: ' + str(hexagons_dict[polyline[0]]['cnt_client']),
                                          tooltip = 'Кол-во жителей: ' + str(hexagons_dict[polyline[0]]['cnt_client']))
            m.add_child(my_Polyline)
    return m   

In [ ]:
hexes_9 = list(set(census_h3['h3_id']))
hexes_8 = list(set(census_h3['h3_id_8']))
hexes_7 = list(set(census_h3['h3_id_7']))
hexes_6 = list(set(census_h3['h3_id_6']))
hexes_5 = list(set(census_h3['h3_id_5']))
hexes_4 = list(set(census_h3['h3_id_4']))
hexes_3 = list(set(census_h3['h3_id_3']))

In [ ]:
f = 'files/home_hexes_agg.xlsx'
hexagons_dict = xlsx_to_dict(f)

In [62]:
#h = ['8910dc93603ffff','8910dc9360fffff','8910dc9361bffff']
m = visualize_hexagons(hexagons_dict)
display(m)